Notre projet consiste à récupérer une très lourde base de données statistiques sur les matchs de football obtenue à partir de fbref.com. Ici, nous allons nous focaliser sur l'ensemble du dépôt pour comprendre comment il est construit.

Importons dans un premier temps l'ensemble des fonctions nécéssaires à l'utilisation de la base de donnés.

In [41]:
from fonctions_preprocess import *
from scrapping import *
pd.set_option('display.max_columns', None)



Notre scrapping permet de récupérer une base brute obenu à partir de fbref.com, pour l'utiliser il suffit d'éxecuter a fonction "..." et d'indiquer en paramètres le nombre de saisons que l'on veut scrapper.
Ici nous nous limiterons à 6 saisons, car au dela il n'ya plus autant de données statistiques disponibles.

Voici un exemple d'utilisation de la fonction:

In [42]:
#soccer_data = scrape_ligue1_data(nb_saisons=6)

Nous allons commencer par analyser la base de données brute en examinant les résultats de notre processus de scrapping (ici nous préférons l'importer car le scrapping est assez couteux en temps (30mn environ pour 6 saisons))

In [43]:
soccer_data = pd.read_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/dynamic_soccer_database.csv", index_col=0)
soccer_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss_x,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Poss_y,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist
0,2018-07-26,17:45,Europa Lg,Second qualifying round,Thu,Away,W,1,0,lv FK Ventspils,NaN,NaN,NaN,3055.0,Benoît Costil,5-3-2,Sergey Lapochkin,Match Report,Leg 1 of 2,Bordeaux,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-08-02,20:45,Europa Lg,Second qualifying round,Thu,Home,W,2,1,lv FK Ventspils,NaN,NaN,NaN,15863.0,Benoît Costil,4-3-3,Hugo Miguel,Match Report,Leg 2 of 2; Bordeaux won,Bordeaux,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-08-04,20:00,Trophée des Champions,Trophée des Champions,Sat,Neutral,L,0,4,Paris S-G,NaN,NaN,27.0,41237.0,Andrea Raggi,4-3-3,Ruddy Buquet,Match Report,NaN,Monaco,0,8.0,2.0,25.0,0.00,0.00,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,4.0,3.0,42.9,0.0,NaN,NaN,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-08-04,20:00,Trophée des Champions,Trophée des Champions,Sat,Neutral,W,4,0,Monaco,NaN,NaN,73.0,41237.0,Thiago Silva,4-3-3,Ruddy Buquet,Match Report,NaN,Paris Saint Germain,4,14.0,7.0,50.0,0.29,0.57,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,100.0,1.0,NaN,NaN,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-08-09,20:00,Europa Lg,Third qualifying round,Thu,Away,W,3,1,ua FC Mariupol,NaN,NaN,NaN,6587.0,Benoît Costil,4-3-3,Ivan Bebek,Match Report,Leg 1 of 2,Bordeaux,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

Nous allons maintenant essayer de mettre en oeuvre la base de données, dans un premier temps nous allons renommer les colonnes par des noms plsu digestes, puis nous allons supprimer certaines colonnes à notre sens peu utile (exemple: distance des passes du gardiens).

Un rapide .describe() nous permet d'observer quelques statistiques pertinentes sur nos bases de données, par exemple en moyenne une équipe tire 12 fois par match, et le max de tirs sur un match est de 38 à l'instant ou nous analysons (27/12/2023).

In [44]:
soccer_data = renommer_colonnes(soccer_data)
soccer_data.describe()


,xG,xGA,Poss_x,Attendance,Total Shots,Shots on Target,Shots on Target %,Goals per Shot,Goals per Shot on Target,Average Shot Distance,Free Kicks Taken,Penalty Kicks Scored,Expected Goals,Non-Penalty Expected Goals,Non-Penalty Expected Goals per Shot,Goal Difference vs Expected Goals,Non-Penalty Goal Difference vs Expected Goals,Poss_y,Total Touches,Touches in Defensive Penalty Area,Touches in Defensive Third,Touches in Midfield Third,Touches in Attacking Third,Touches in Attacking Penalty Area,Dribbles Attempted,Successful Dribbles,Successful Dribble %,Dribbles Tackled,Dribble Tackle %,Total Carries,Total Carry Distance,Progressive Carry Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Carries Miscontrolled,Carries Dispossessed,Tackles,Tackles Won,Tackles in Defensive Third,Tackles in Midfield Third,Tackles in Attacking Third,Dribblers Tackled,Total Dribbles Against,Defensive Dribblers Win %,Defensive Challenges Lost,Total Blocks,Shot Blocks,Pass Blocks,Interceptions,Tackles Plus Interceptions,Clearances,Errors Leading to Goal,Passes Completed,Passes Attempted,Pass Completion %,Total Pass Distance,Progressive Pass Distance,Short Passes Completed,Short Passes Attempted,Short Pass Completion %,Medium Passes Completed,Medium Passes Attempted,Medium Pass Completion %,Long Passes Completed,Long Passes Attempted,Long Pass Completion %,Assists,Expected Assists Goals,Expected Assists,Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Shots on Target Against,Keeper Saves,Keeper Save Percentage,Clean Sheets,Penalty Kicks Against,Penalty Kicks Against Saved,Penalty Kicks Against Missed
count,4164.000000,4164.000000,4427.000000,3656.000000,4491.000000,4491.000000,4465.000000,4465.000000,4336.000000,4163.000000,4164.000000,4555.000000,4164.000000,4164.000000,4162.000000,4164.000000,4164.000000,4427.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4491.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4491.000000,4555.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4553.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4491.000000,4491.000000,4311.000000,4550.000000,4555.000000,4554.000000,4554.000000
mean,1.340874,1.326777,50.403659,22974.490153,12.003785,4.083500,34.603113,0.109406,0.295918,18.442277,0.524736,0.147091,1.340874,1.199592,0.102150,-0.030115,-0.033405,50.403659,612.134726,57.947887,187.685879,294.829491,136.079251,19.139529,18.211816,9.693324,53.163064,7.960615,43.999976,377.941883,2072.098703,1039.060999,18.256004,14.315082,4.159222,16.123439,9.493756,17.329971,10.272100,8.268492,6.964217,2.097262,7.858309,17.382325,45.493684,9.524015,10.121278,2.882325,7.238953,10.902472,25.915477,17.981748,0.321326,401.605908,501.665466,78.949784,7135.310519,2582.046830,178.429635,203.223583,86.934030,166.744957,193.926513,84.870605,43.321085,75.991354,56.761407,0.905996,0.933069,0.890154,8.962776,30.278818,7.060759,1.908021,39.619116,4.109998,2.802271,70.343122,0.279121,0.141164,0.026350,0.009003
std,0.776610,0.771758,10.579841,15605.170456,5.046663,2.399967,16.877885,0.126673,0.272288,3.291863,0.739154,0.381111,0.776610,0.701696,0.046741,0.963577,0.952411,10.579841,112.617056,16.359137,38.415629,80.619462,50.662396,8.626473,6.277114,4.216156,14.155097,3.485378,13.404847,117.921547,639.390087,336.355183,6.781570,6.346368,2.602842,4.641367,3.861300,5.152365,3.832097,3.741780,3.102527,1.683996,3.468828,6.042235,13.963476,4.163619,3.755363,2.014739,3.067706,4.325026,10.364148,8.369681,0.584051,118.52

Nous allons maintenant procéder à un premier prétraitement de la base pour la rendre utilisable :

- Transformation de la date et de l'heure du match en une seule variable.
- Création d'une variable "Saison" qui indique la saison pendant laquelle a eu lieu le match.
- Mapping des équipes en raison des différences de noms entre "Opponent" et "Team" (3 équipes différentes).
- Conservation uniquement des matchs de Ligue 1 en utilisant la colonne "Comp".
- Extraction de la journée du match, par exemple, "Matchweek 3" devient simplement "3".
- Nettoyage de la colonne 'Formation' en remplaçant le caractère '◆' par une chaîne vide dans les lignes qui ont ce caractère en plus
- Renommage de la colonne "Poss_x" en "Poss" et suppression de la colonne "Poss_y", si elles existent (dû au scrapping)
- Création de la variable "MatchID" en combinant les colonnes "Team" et "Opponent".



In [45]:
mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
    }

soccer_data = preprocess_initial(soccer_data, mapping_equipe)
soccer_data.head()

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Total Shots,Shots on Target,Shots on Target %,Goals per Shot,Goals per Shot on Target,Average Shot Distance,Free Kicks Taken,Penalty Kicks Scored,Expected Goals,Non-Penalty Expected Goals,Non-Penalty Expected Goals per Shot,Goal Difference vs Expected Goals,Non-Penalty Goal Difference vs Expected Goals,Total Touches,Touches in Defensive Penalty Area,Touches in Defensive Third,Touches in Midfield Third,Touches in Attacking Third,Touches in Attacking Penalty Area,Dribbles Attempted,Successful Dribbles,Successful Dribble %,Dribbles Tackled,Dribble Tackle %,Total Carries,Total Carry Distance,Progressive Carry Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Carries Miscontrolled,Carries Dispossessed,Tackles,Tackles Won,Tackles in Defensive Third,Tackles in Midfield Third,Tackles in Attacking Third,Dribblers Tackled,Total Dribbles Against,Defensive Dribblers Win %,Defensive Challenges Lost,Total Blocks,Shot Blocks,Pass Blocks,Interceptions,Tackles Plus Interceptions,Clearances,Errors Leading to Goal,Passes Completed,Passes Attempted,Pass Completion %,Total Pass Distance,Progressive Pass Distance,Short Passes Completed,Short Passes Attempted,Short Pass Completion %,Medium Passes Completed,Medium Passes Attempted,Medium Pass Completion %,Long Passes Completed,Long Passes Attempted,Long Pass Completion %,Assists,Expected Assists Goals,Expected Assists,Key Passes,Passes into Final Third,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Shots on Target Against,Keeper Saves,Keeper Save Percentage,Clean Sheets,Penalty Kicks Against,Penalty Kicks Against Saved,Penalty Kicks Against Missed,Saison,MatchID
5,2018-08-10 20:45:00,Ligue 1,1,Fri,Home,W,4,0,Toulouse,3.2,0.3,60.0,60756.0,Dimitri Payet,4-2-3-1,Ruddy Buquet,Match Report,NaN,Marseille,22.0,9.0,40.9,0.14,0.33,18.6,2.0,1,3.2,2.5,0.11,0.8,0.5,660.0,45.0,205.0,313.0,153.0,30.0,13.0,6.0,46.2,7.0,53.8,438.0,2287.0,1333.0,31.0,26.0,8.0,15.0,12.0,29.0,20.0,11.0,11.0,7.0,15.0,26.0,57.7,11.0,8.0,0.0,8.0,7.0,36,14.0,0.0,448.0,542.0,82.7,7324.0,2540.0,210.0,232.0,90.5,147.0,167.0,88.0,54.0,87.0,62.1,2.0,1.7,0.9,14.0,18.0,6.0,3.0,29.0,1.0,1.0,100.0,1.0,0,0.0,0.0,2018-2019,Marseille_Toulouse
6,2018-08-10 20:45:00,Ligue 1,1,Fri,Away,L,0,4,Marseille,0.3,3.2,40.0,60756.0,Max Gradel,4-2-3-1,Ruddy Buquet,Match Report,NaN,Toulouse,5.0,1.0,20.0,0.00,0.00,20.3,0.0,0,0.3,0.3,0.06,-0.3,-0.3,496.0,71.0,182.0,225.0,93.0,11.0,26.0,11.0,42.3,15.0,57.7,303.0,1727.0,887.0,17.0,13.0,2.0,27.0,14.0,19.0,12.0,13.0,3.0,3.0,7.0,13.0,53.8,6.0,8.0,3.0,5.0,15.0,34,22.0,1.0,281.0,360.0,78.1,5171.0,2262.0,112.0,130.0,86.2,131.0,156.0,84.0,31.0,56.0,55.4,0.0,0.2,0.6,3.0,16.0,4.0,2.0,32.0,10.0,6.0,70.0,0.0,1,0.0,0.0,2018-2019,Toulouse_Marseille
7,2018-08-11 20:00:00,Ligue 1,1,Sat,Home,L,3,4,Nîmes,2.8,1.5,57.0,9534.0,Ismaël Traoré,4-2-3-1,Antony Gautier,Match Report,NaN,Angers,20.0,7.0,35.0,0.15,0.43,17.6,1.0,0,2.8,2.8,0.14,0.2,0.2,617.0,32.0,136.0,354.0,134.0,24.0,22.0,13.0,59.1,9.0,40.9,294.0,1442.0,723.0,13.0,16.0,4.0,14.0,12.0,19.0,15.0,5.0,12.0,2.0,10.0,18.0,55.6,8.0,11.0,4.0,7.0,3.0,22,16.0,1.0,381.0,506.0,75.3,6423.0,2503.0,179.0,207.0,86.5,151.0,186.0,81.2,37.0,79.0,46.8,2.0,2.0,1.4,12.0,25.0,6.0,0.0,44.0,4.0,1.0,0.0,0.0,0,0.0,0.0,2018-2019,Angers_Nîmes
8,2018-08-11 20:00:00,Ligue 1,1,Sat,Away,W,2,1,Montpellier,2.0,1.3,46.0,12765.0,Júlio Tavares,3-4-3,Florent Batta,Match Report,NaN,Dijon,9.0,3.0,33.3,0.22,0.67,17.2,1.0,0,2.0,1.3,0.14,0.0,0.7,522.0,67.0,182.0,265.0,80.0,14.0,14.0,9.0,64.3,5.0,35.7,302.0,1789.0,868.0,17.0,13.0,5.0,14.0,10.0,16.0,10.0,8.0,8.0,0.0,0.0,10.0,0.0,10.0,9.0,2.0,7.0,12.0,28,29.0,0.0,323.0,423.0,76.4,6289.0,2948.0,135.0,158.0,85.4,126.0,150.0,84.0,52.0,95.0,54.7,1.0,0.6,0.4,6.0,20.0,3.0,1.0,28.0,7.0,6.0,85.7,0.0,0,0.0,0.0,2018-2019,Dijon_Montpellier
9,2018-08-11 20:00:00,Ligue 1,1,Sat,Away,L,1,2,Saint-Étienne,1.6,0.8,57.0,2

Par choix personnel, nous avons décidé de nous restreindre qu'à l'étude de certainnes variables, étant donné que certainnes sont redondantes et d'autres ne semblent pas pertinentes dans la prédictions de résultats de football.
De plus, étant donné que nous observons des matchs de chaque côté, c'est à dire que pour un même match nous avons Paris contre Lyon et une autre ligne Lyon contre Paris car les statistiques sont propres à l'équipe dans "Team", il fallait réduire el nombre de colonnes pour fuisonner les lignes et ne pas avoir trop de colonnes par rapport au nombre de lignes pour éviter le surapprentissage.

In [46]:
soccer_data = columns_to_keep(soccer_data)
soccer_data.head()

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,MatchID,Saison,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Total Shots,Shots on Target,Shots on Target %,Goals per Shot,Total Touches,Touches in Defensive Penalty Area,Touches in Defensive Third,Touches in Midfield Third,Touches in Attacking Third,Touches in Attacking Penalty Area,Dribbles Attempted,Successful Dribbles,Successful Dribble %,Total Carries,Total Carry Distance,Progressive Carry Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Tackles,Tackles Won,Tackles in Defensive Third,Tackles in Midfield Third,Tackles in Attacking Third,Dribblers Tackled,Total Dribbles Against,Defensive Dribblers Win %,Interceptions,Errors Leading to Goal,Key Passes,Passes Completed,Passes Attempted,Passes into Final Third,Progressive Passes,Shots on Target Against,Keeper Saves,Keeper Save Percentage
5,2018-08-10 20:45:00,Ligue 1,1,Fri,Home,W,4,0,Toulouse,Marseille_Toulouse,2018-2019,60756.0,Dimitri Payet,4-2-3-1,Ruddy Buquet,Match Report,NaN,Marseille,22.0,9.0,40.9,0.14,660.0,45.0,205.0,313.0,153.0,30.0,13.0,6.0,46.2,438.0,2287.0,1333.0,31.0,26.0,8.0,29.0,20.0,11.0,11.0,7.0,15.0,26.0,57.7,7.0,0.0,14.0,448.0,542.0,18.0,29.0,1.0,1.0,100.0
6,2018-08-10 20:45:00,Ligue 1,1,Fri,Away,L,0,4,Marseille,Toulouse_Marseille,2018-2019,60756.0,Max Gradel,4-2-3-1,Ruddy Buquet,Match Report,NaN,Toulouse,5.0,1.0,20.0,0.00,496.0,71.0,182.0,225.0,93.0,11.0,26.0,11.0,42.3,303.0,1727.0,887.0,17.0,13.0,2.0,19.0,12.0,13.0,3.0,3.0,7.0,13.0,53.8,15.0,1.0,3.0,281.0,360.0,16.0,32.0,10.0,6.0,70.0
7,2018-08-11 20:00:00,Ligue 1,1,Sat,Home,L,3,4,Nîmes,Angers_Nîmes,2018-2019,9534.0,Ismaël Traoré,4-2-3-1,Antony Gautier,Match Report,NaN,Angers,20.0,7.0,35.0,0.15,617.0,32.0,136.0,354.0,134.0,24.0,22.0,13.0,59.1,294.0,1442.0,723.0,13.0,16.0,4.0,19.0,15.0,5.0,12.0,2.0,10.0,18.0,55.6,3.0,1.0,12.0,381.0,506.0,25.0,44.0,4.0,1.0,0.0
8,2018-08-11 20:00:00,Ligue 1,1,Sat,Away,W,2,1,Montpellier,Dijon_Montpellier,2018-2019,12765.0,Júlio Tavares,3-4-3,Florent Batta,Match Report,NaN,Dijon,9.0,3.0,33.3,0.22,522.0,67.0,182.0,265.0,80.0,14.0,14.0,9.0,64.3,302.0,1789.0,868.0,17.0,13.0,5.0,16.0,10.0,8.0,8.0,0.0,0.0,10.0,0.0,12.0,0.0,6.0,323.0,423.0,20.0,28.0,7.0,6.0,85.7
9,2018-08-11 20:00:00,Ligue 1,1,Sat,Away,L,1,2,Saint-Étienne,Guingamp_Saint-Étienne,2018-2019,26006.0,Christophe Kerbrat,4-2-3-1,Amaury Delerue,Match Report,NaN,Guingamp,10.0,4.0,40.0,0.00,623.0,63.0,151.0,348.0,131.0,22.0,13.0,8.0,61.5,317.0,1996.0,869.0,23.0,17.0,3.0,10.0,7.0,3.0,5.0,2.0,2.0,5.0,40.0,5.0,0.0,9.0,399.0,529.0,28.0,46.0,6.0,4.0,66.7


Maintenant il est important d'apporter des colonnes qui sont manquantes et qui sont déterminantes pour faire de la prévisions. 

- Le classement d'une équipe est manquant à chaque observation 
- Son nombre de points, de buts marqués/encaissés cumulés durant la saison jusqu'à la date du amtch est manquant et constitue une caractéristique majeur dans la prevision
- De plus il serait intéressant de regarder les anciens résultats entre les deux équipes qui s'affrontent, ainsi que le nombre de rencontres déjà effectué entre ces deux équipes.


In [47]:
soccer_data = preprocess_variables(soccer_data)
soccer_data.head()

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,MatchID,Saison,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Total Shots,Shots on Target,Shots on Target %,Goals per Shot,Total Touches,Touches in Defensive Penalty Area,Touches in Defensive Third,Touches in Midfield Third,Touches in Attacking Third,Touches in Attacking Penalty Area,Dribbles Attempted,Successful Dribbles,Successful Dribble %,Total Carries,Total Carry Distance,Progressive Carry Distance,Progressive Carries,Carries into Final Third,Carries into Penalty Area,Tackles,Tackles Won,Tackles in Defensive Third,Tackles in Midfield Third,Tackles in Attacking Third,Dribblers Tackled,Total Dribbles Against,Defensive Dribblers Win %,Interceptions,Errors Leading to Goal,Key Passes,Passes Completed,Passes Attempted,Passes into Final Third,Progressive Passes,Shots on Target Against,Keeper Saves,Keeper Save Percentage,GD,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,Classement,Past_Matches,IsWin,IsDraw,IsLoss,CumulativeWins,CumulativeDraws,CumulativeLosses
14,2018-08-12 15:00:00,Ligue 1,1,Sun,Away,L,0,2,Lyon,Amiens_Lyon,2018-2019,48263.0,Régis Gurtner,4-3-3,Franck Schneider,Match Report,NaN,Amiens,10.0,1.0,10.0,0.00,414.0,49.0,152.0,195.0,73.0,6.0,13.0,8.0,61.5,198.0,1296.0,710.0,14.0,10.0,2.0,23.0,8.0,14.0,7.0,2.0,12.0,28.0,42.9,14.0,0.0,7.0,216.0,301.0,18.0,26.0,7.0,5.0,71.4,-2,0,0,-2,0,2,15,0,0,0,1,0,0,1
36,2018-08-18 20:00:00,Ligue 1,2,Sat,Home,L,1,2,Montpellier,Amiens_Montpellier,2018-2019,10402.0,Prince-Désir Gouano,4-2-3-1,Karim Abed,Match Report,NaN,Amiens,10.0,1.0,10.0,0.00,609.0,35.0,156.0,329.0,130.0,19.0,8.0,7.0,87.5,388.0,2447.0,1283.0,34.0,10.0,2.0,14.0,7.0,9.0,4.0,1.0,4.0,10.0,40.0,13.0,0.0,5.0,415.0,524.0,34.0,38.0,3.0,1.0,33.3,-1,0,0,-3,1,4,17,0,0,0,1,0,0,1
53,2018-08-25 20:00:00,Ligue 1,3,Sat,Home,W,4,1,Reims,Amiens_Reims,2018-2019,10709.0,Régis Gurtner,4-2-3-1,Florent Batta,Match Report,NaN,Amiens,14.0,8.0,57.1,0.29,658.0,37.0,179.0,340.0,147.0,19.0,13.0,8.0,61.5,431.0,2343.0,1200.0,30.0,18.0,5.0,14.0,7.0,4.0,5.0,5.0,9.0,17.0,52.9,7.0,0.0,11.0,489.0,581.0,32.0,47.0,3.0,2.0,66.7,3,3,3,0,5,5,14,0,1,0,0,1,0,0
72,2018-09-02 15:00:00,Ligue 1,4,Sun,Away,D,0,0,Saint-Étienne,Amiens_Saint-Étienne,2018-2019,30146.0,Régis Gurtner,4-3-3,Thomas Léonard,Match Report,NaN,Amiens,9.0,2.0,22.2,0.00,454.0,75.0,197.0,181.0,82.0,10.0,12.0,6.0,50.0,211.0,1247.0,672.0,14.0,7.0,3.0,6.0,3.0,4.0,1.0,1.0,1.0,7.0,14.3,14.0,0.0,9.0,232.0,327.0,29.0,25.0,5.0,5.0,100.0,0,1,4,0,5,5,13,0,0,1,0,0,1,0
97,2018-09-15 20:00:00,Ligue 1,5,Sat,Home,L,2,3,Lille,Amiens_Lille,2018-2019,11569.0,Régis Gurtner,4-3-3,Nicolas Rainville,Match Report,NaN,Amiens,7.0,3.0,42.9,0.29,521.0,53.0,178.0,274.0,73.0,10.0,5.0,4.0,80.0,253.0,1327.0,655.0,11.0,2.0,2.0,19.0,14.0,11.0,6.0,2.0,7.0,12.0,58.3,9.0,0.0,6.0,308.0,434.0,17.0,32.0,9.0,6.0,88.9,-1,0,4,-1,7,8,18,0,0,0,1,0,0,1


Nous avons donc ici une très bonne base pour travailler, nous recenserons toutes les statistiques descriptives dans l'interface afin qu'elles soient facilement accesible par l'utilisateur en quelque clics étant donné l'importance pour un amateur de football de rcueillir les derneres statistiques sur les équipes à des buts personnels ou de préviisons.


Exemple d'utilisation de de l'interface :
(attention, il faut executer la fonction dans un environnement non cloud qui supporte tkinter)

Il est important de noter que nous devons créer un modèle de prédictions basé des données temporelles, c'est à dire décalé car nous ne disposons pas des statsitques détaillés d'un match avant sa réalisation.

Nous allons donc créer des statistques laggés de une période sur plusieurs variables :
- Classement
- Points
- Buts marqués/encaissés cumulés

Puis nous allons ajouter la variable qui recense les dernieres informations sur les confrontations entre deux équipes décalés de l'info disponible au dernier match.


Enfin, nous allons générer des statistiques roulantes par saison, prenons l'exemple des tirs cadrés par équipes, nous allons créer des moyennes mobiles sur des fênetres roulantes en respectant les performances des équipes sur une saison. En effet, il est plus pertinent d'étudier les performances d'une équipe le long d'une saisons que sur l'ensemble des saisons.


In [48]:
soccer_data = preparation_model(soccer_data)
soccer_data.head()

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,MatchID,Saison,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,GD,Points,Past_Matches,IsWin,IsDraw,IsLoss,Points_Cum_Lag1,GD_Cum_Lag1,GF_Cum_Lag1,GA_Cum_Lag1,Classement_Lag1,CumulativeWins_Lag1,CumulativeDraws_Lag1,CumulativeLosses_Lag1,Moyenne_Total Shots_Lag,Moyenne_Shots on Target_Lag,Moyenne_Shots on Target %_Lag,Moyenne_Goals per Shot_Lag,Moyenne_Total Touches_Lag,Moyenne_Touches in Defensive Penalty Area_Lag,Moyenne_Touches in Defensive Third_Lag,Moyenne_Touches in Midfield Third_Lag,Moyenne_Touches in Attacking Third_Lag,Moyenne_Touches in Attacking Penalty Area_Lag,Moyenne_Dribbles Attempted_Lag,Moyenne_Successful Dribbles_Lag,Moyenne_Successful Dribble %_Lag,Moyenne_Total Carries_Lag,Moyenne_Total Carry Distance_Lag,Moyenne_Progressive Carry Distance_Lag,Moyenne_Progressive Carries_Lag,Moyenne_Carries into Final Third_Lag,Moyenne_Carries into Penalty Area_Lag,Moyenne_Tackles_Lag,Moyenne_Tackles Won_Lag,Moyenne_Tackles in Defensive Third_Lag,Moyenne_Tackles in Midfield Third_Lag,Moyenne_Tackles in Attacking Third_Lag,Moyenne_Dribblers Tackled_Lag,Moyenne_Total Dribbles Against_Lag,Moyenne_Defensive Dribblers Win %_Lag,Moyenne_Interceptions_Lag,Moyenne_Errors Leading to Goal_Lag,Moyenne_Key Passes_Lag,Moyenne_Passes Completed_Lag,Moyenne_Passes Attempted_Lag,Moyenne_Passes into Final Third_Lag,Moyenne_Progressive Passes_Lag,Moyenne_Shots on Target Against_Lag,Moyenne_Keeper Saves_Lag,Moyenne_Keeper Save Percentage_Lag
0,2018-08-12 15:00:00,Ligue 1,1,Sun,Away,L,0,2,Lyon,Amiens_Lyon,2018-2019,48263.0,Régis Gurtner,4-3-3,Franck Schneider,Match Report,NaN,Amiens,-2,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-08-18 20:00:00,Ligue 1,2,Sat,Home,L,1,2,Montpellier,Amiens_Montpellier,2018-2019,10402.0,Prince-Désir Gouano,4-2-3-1,Karim Abed,Match Report,NaN,Amiens,-1,0,0,0,0,1,0.0,-2.0,0.0,2.0,15.0,NaN,NaN,NaN,10.000000,1.000000,10.000,0.000000,414.000000,49.000000,152.000000,195.00,73.000000,6.000000,13.000000,8.000000,61.500000,198.0,1296.000000,710.000000,14.0,10.000000,2.0,23.00,8.000000,14.00,7.000000,2.000000,12.000000,28.000000,42.900000,14.000000,0.0,7.000000,216.000000,301.000000,18.00,26.0,7.000000,5.000000,71.400000
2,2018-08-25 20:00:00,Ligue 1,3,Sat,Home,W,4,1,Reims,Amiens_Reims,2018-2019,10709.0,Régis Gurtner,4-2-3-1,Florent Batta,Match Report,NaN,Amiens,3,3,0,1,0,0,0.0,-3.0,1.0,4.0,17.0,NaN,NaN,NaN,10.000000,1.000000,10.000,0.000000,511.500000,42.000000,154.000000,262.00,101.500000,12.500000,10.500000,7.500000,74.500000,293.0,1871.500000,996.500000,24.0,10.000000,2.0,18.50,7.500000,11.50,5.500000,1.500000,8.000000,19.000000,41.450000,13.500000,0.0,6.000000,315.500000,412.500000,26.00,32.0,5.000000,3.000000,52.350000
3,2018-09-02 15:00:00,Ligue 1,4,Sun,Away,D,0,0,Saint-Étienne,Amiens_Saint-Étienne,2018-2019,30146.0,Régis Gurtner,4-3-3,Thomas Léonard,Match Report,NaN,Amiens,0,1,0,0,1,0,3.0,0.0,5.0,5.0,14.0,NaN,NaN,NaN,11.333333,3.333333,25.700,0.096667,560.333333,40.333333,162.333333,288.00,116.666667,14.666667,11.333333,7.666667,70.166667,339.0,2028.666667,1064.333333,26.0,12.666667,3.0,17.00,7.333333,9.00,5.333333,2.666667,8.333333,18.333333,45.266667,11.333333,0.0,7.666667,373.333333,468.666667,28.00,37.0,4.333333,2.666667,57.133333
4,2018-09-15 20:00:00,Ligue 1,5,Sat,Home,L,2,3,Lille,Amiens_Lille,2018-2019,11569.0,Régis Gurtner,4-3-3,Nicolas Rainville,Match Report,NaN,Amiens,-1,0,0,0,0,1,4.0,0.0,5.0,5.0,13.0,NaN,NaN,NaN,10.750000,3.000000,24.825,0.072500,533.750000,49.000000,171.000000,261.25,108.000000,13.500000,11.500000,7.250000,65.125000,307.0,1833.250000,966.250000,23.0,11.250000,3.0,14.25,6.250000,7.75,4.250000,2.250000,6.500000,15.500000,37.525000,12.000000,0.0,8.000000,338.000000,433.250000,28.25,34.0,4.500000,3.250000,67.850000


À ce stade il est déterminant de relier les deux lignes d'un même match en une seule ligne afin de capter l'ensemble de l'infirmation disonible pour le sdeux équipes pour prédire au mieux le résultat.

Nous allons donc créer un identifiant unique basé sur la date et sur les deux noms des équipes qui s'affrotnent afin de pouvoir merger les deux lignes. De plus on prendra en compte comme critère de tri l'équipe qui joue à dmicie et celle qui joue à l'exterieur.

Ensuite nous récréeons la colonne Result qui prendre donc 3 valeurs et qui sera notre variable à prédire.
- W_Home
- W_Away
- D (draw)

La base est donc divisé en deux en termes de lignes et le nombre de colonne par ligne est multiplié par deux.

In [49]:
soccer_data = preprocess_data(soccer_data)
soccer_data.head()

,DateTime,Comp,Round,Day,Venue_Home,Result_Home,GF_Home,GA_Home,Opponent_Home,MatchID,Saison,Attendance_Home,Captain_Home,Formation_Home,Referee,Match Report,Notes,Team Home,GD_Home,Points_Home,Past_Matches_Home,IsWin_Home,IsDraw_Home,IsLoss_Home,Points_Cum_Lag1_Home,GD_Cum_Lag1_Home,GF_Cum_Lag1_Home,GA_Cum_Lag1_Home,Classement_Lag1_Home,CumulativeWins_Lag1_Home,CumulativeDraws_Lag1_Home,CumulativeLosses_Lag1_Home,Moyenne_Total Shots_Lag_Home,Moyenne_Shots on Target_Lag_Home,Moyenne_Shots on Target %_Lag_Home,Moyenne_Goals per Shot_Lag_Home,Moyenne_Total Touches_Lag_Home,Moyenne_Touches in Defensive Penalty Area_Lag_Home,Moyenne_Touches in Defensive Third_Lag_Home,Moyenne_Touches in Midfield Third_Lag_Home,Moyenne_Touches in Attacking Third_Lag_Home,Moyenne_Touches in Attacking Penalty Area_Lag_Home,Moyenne_Dribbles Attempted_Lag_Home,Moyenne_Successful Dribbles_Lag_Home,Moyenne_Successful Dribble %_Lag_Home,Moyenne_Total Carries_Lag_Home,Moyenne_Total Carry Distance_Lag_Home,Moyenne_Progressive Carry Distance_Lag_Home,Moyenne_Progressive Carries_Lag_Home,Moyenne_Carries into Final Third_Lag_Home,Moyenne_Carries into Penalty Area_Lag_Home,Moyenne_Tackles_Lag_Home,Moyenne_Tackles Won_Lag_Home,Moyenne_Tackles in Defensive Third_Lag_Home,Moyenne_Tackles in Midfield Third_Lag_Home,Moyenne_Tackles in Attacking Third_Lag_Home,Moyenne_Dribblers Tackled_Lag_Home,Moyenne_Total Dribbles Against_Lag_Home,Moyenne_Defensive Dribblers Win %_Lag_Home,Moyenne_Interceptions_Lag_Home,Moyenne_Errors Leading to Goal_Lag_Home,Moyenne_Key Passes_Lag_Home,Moyenne_Passes Completed_Lag_Home,Moyenne_Passes Attempted_Lag_Home,Moyenne_Passes into Final Third_Lag_Home,Moyenne_Progressive Passes_Lag_Home,Moyenne_Shots on Target Against_Lag_Home,Moyenne_Keeper Saves_Lag_Home,Moyenne_Keeper Save Percentage_Lag_Home,Venue_Away,Result_Away,GF_Away,GA_Away,Opponent_Away,Attendance_Away,Captain_Away,Formation_Away,Team Away,GD_Away,Points_Away,Past_Matches_Away,IsWin_Away,IsDraw_Away,IsLoss_Away,Points_Cum_Lag1_Away,GD_Cum_Lag1_Away,GF_Cum_Lag1_Away,GA_Cum_Lag1_Away,Classement_Lag1_Away,CumulativeWins_Lag1_Away,CumulativeDraws_Lag1_Away,CumulativeLosses_Lag1_Away,Moyenne_Total Shots_Lag_Away,Moyenne_Shots on Target_Lag_Away,Moyenne_Shots on Target %_Lag_Away,Moyenne_Goals per Shot_Lag_Away,Moyenne_Total Touches_Lag_Away,Moyenne_Touches in Defensive Penalty Area_Lag_Away,Moyenne_Touches in Defensive Third_Lag_Away,Moyenne_Touches in Midfield Third_Lag_Away,Moyenne_Touches in Attacking Third_Lag_Away,Moyenne_Touches in Attacking Penalty Area_Lag_Away,Moyenne_Dribbles Attempted_Lag_Away,Moyenne_Successful Dribbles_Lag_Away,Moyenne_Successful Dribble %_Lag_Away,Moyenne_Total Carries_Lag_Away,Moyenne_Total Carry Distance_Lag_Away,Moyenne_Progressive Carry Distance_Lag_Away,Moyenne_Progressive Carries_Lag_Away,Moyenne_Carries into Final Third_Lag_Away,Moyenne_Carries into Penalty Area_Lag_Away,Moyenne_Tackles_Lag_Away,Moyenne_Tackles Won_Lag_Away,Moyenne_Tackles in Defensive Third_Lag_Away,Moyenne_Tackles in Midfield Third_Lag_Away,Moyenne_Tackles in Attacking Third_Lag_Away,Moyenne_Dribblers Tackled_Lag_Away,Moyenne_Total Dribbles Against_Lag_Away,Moyenne_Defensive Dribblers Win %_Lag_Away,Moyenne_Interceptions_Lag_Away,Moyenne_Errors Leading to Goal_Lag_Away,Moyenne_Key Passes_Lag_Away,Moyenne_Passes Completed_Lag_Away,Moyenne_Passes Attempted_Lag_Away,Moyenne_Passes into Final Third_Lag_Away,Moyenne_Progressive Passes_Lag_Away,Moyenne_Shots on Target Against_Lag_Away,Moyenne_Keeper Saves_Lag_Away,Moyenne_Keeper Save Percentage_Lag_Away,Result
0,2018-08-18 20:00:00,Ligue 1,2,Sat,Home,L,1,2,Montpellier,2018-08-18 20:00:00-Amiens-vs-Montpellier,2018-2019,10402.0,Prince-Désir Gouano,4-2-3-1,Karim Abed,Match Report,NaN,Amiens,-1,0,0,0,0,1,0.0,-2.0,0.0,2.0,15.0,NaN,NaN,NaN,10.000000,1.000,10.0000,0.000000,414.000000,49.000000,152.0,195.000000,73.000000,6.000000,13.000000,8.000000,61.500000,198.000000,1296.000000,710.000,14.000000,10.000,2.000000,23.000000,8.000000,14.000

À partir d'un modèle de random forest nous avons essayé de prédire au mieux la variable cible "Result", néanmoins il est tout à fait possible d'utiliser d'autres algortihmes qui peuvent utiliser des variables catégorielles en prévision.
Notre modèle estime "Result" à partir d'une date cutoff, qui permet de séparer l'échantillon d'entrainement et de test.


De plus, notre modèle estime les probabilités pour chaque événèment prédits, par exemple si une victoire de l'équipe à domicile est prédites, il estime également la probabilité que l'évènement arrive réellement. Ceci suera un trsè bonne inidcateur pour les utilisateurs de notre interface.

Voyons un exemple :

In [51]:
prédiction = modelisation(soccer_data, cutoff_date=datetime(2023, 9, 1))

Regardons les matchs avec une probabilité de prédictions supérieurs à 60% et comaprons aux résultats réels.

In [53]:
prédiction[prédiction["Prediction_Probability"] >= 0.6]

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,Predicted_Result,Prediction_Probability,MatchID
1809,2023-09-17 15:00:00,Ligue 1,2023-2024,5,Sun,Clermont Foot,Nantes,W_Away,W_Home,0.65,2023-09-17 15:00:00-Clermont Foot-vs-Nantes
1828,2023-10-28 21:00:00,Ligue 1,2023-2024,10,Sat,Lens,Nantes,W_Home,W_Home,0.66,2023-10-28 21:00:00-Lens-vs-Nantes
1829,2023-11-12 20:45:00,Ligue 1,2023-2024,12,Sun,Lens,Marseille,W_Home,W_Home,0.62,2023-11-12 20:45:00-Lens-vs-Marseille
1831,2023-12-16 21:00:00,Ligue 1,2023-2024,16,Sat,Lens,Reims,W_Home,W_Home,0.64,2023-12-16 21:00:00-Lens-vs-Reims
1835,2023-10-22 15:00:00,Ligue 1,2023-2024,9,Sun,Lille,Brest,W_Home,W_Home,0.63,2023-10-22 15:00:00-Brest-vs-Lille
1856,2023-12-10 17:05:00,Ligue 1,2023-2024,15,Sun,Lyon,Toulouse,W_Home,W_Home,0.64,2023-12-10 17:05:00-Lyon-vs-Toulouse
1877,2023-09-30 21:00:00,Ligue 1,2023-2024,7,Sat,Monaco,Marseille,W_Home,W_Home,0.61,2023-09-30 21:00:00-Marseille-vs-Monaco
1879,2023-11-05 17:05:00,Ligue 1,2023-2024,11,Sun,Monaco,Brest,W_Home,W_Home,0.68,2023-11-05 17:05:00-Brest-vs-Monaco
1880,2023-12-03 15:00:00,Ligue 1,2023-2024,14,Sun,Monaco,Montpellier,W_Home,W_Home,0.70,2023-12-03 15:00:00-Monaco-vs-Montpellier
1951,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Toulouse,Monaco,W_Away,W_Away,0.62,2023-12-20 21:00:00-Monaco-vs-Toulouse


On voit que l'algorithme prédis trsè bien les matchs à haute probabilité., ici un seul match est mal prédit.
Néanmoins, on voit bien qu'il est beaucoup plus facile de prédire une victoire à domicile qu'un match nul ou une victoire à l'extérieur et ce malgré la gestion des catégories dans l'estimation du Random Forest.
